# SQLite Evaluation Queries
This notebook evaluates the equivalent queries against the structured SQLite schema.

In [5]:
import sqlite3
conn = sqlite3.connect("../sql/openfda_base.db")
cursor = conn.cursor()

## Query 1: All reports where `medicinalproduct = 'INFLECTRA'`

In [15]:
query = """
SELECT r.safetyreportid
FROM report r
JOIN drug d ON r.safetyreportid = d.safetyreportid
WHERE d.medicinalproduct = 'INFLECTRA';
"""
cursor.execute(query)
results = cursor.fetchall()
print(f"Found {len(results)} reports.")
what = set()
for row in results:
    what.add(row[0])
print(f"Found {len(what)} unique reports.")

Found 2569 reports.
Found 177 unique reports.


## Query 2: Reports where `patientsex = '2'` and one `reactionoutcome = '5'` exists

In [20]:
query = """
SELECT DISTINCT r.safetyreportid
FROM report r
JOIN patient_optional p ON r.safetyreportid = p.safetyreportid
JOIN reaction re ON r.safetyreportid = re.safetyreportid
WHERE p.patientsex = '2' AND re.reactionoutcome = '5';
"""

cursor.execute(query)
results = cursor.fetchall()
print(f"Found {len(results)} reports.")
what = set()
for row in results:
    what.add(row[0])

Found 1094 reports.


## Query 3: Number of serious reports by `medicinalproduct`

In [17]:
query = """
SELECT d.medicinalproduct, COUNT(*) AS serious_count
FROM report r
JOIN drug d ON r.safetyreportid = d.safetyreportid
WHERE r.serious = '1'
GROUP BY d.medicinalproduct
ORDER BY serious_count DESC;
"""

cursor.execute(query)
results = cursor.fetchall()
print(f"Found {len(results)} rows.")
from pprint import pprint
pprint(results[:5])


Found 7209 rows.
[('INFLECTRA', 2559),
 ('PREDNISONE', 2149),
 ('METHOTREXATE', 1982),
 ('RITUXIMAB', 1388),
 ('ACETAMINOPHEN', 1351)]


## Query 4: Reports that have duplicate references

In [19]:
query = """
SELECT DISTINCT r.safetyreportid
FROM report r
WHERE r.duplicate = '1';
"""

cursor.execute(query)
results = cursor.fetchall()
print(f"Found {len(results)} reports with duplicates.")


Found 11715 reports with duplicates.


## Query 5: Reports with `activesubstancename = 'INFLIXIMAB'`

In [12]:
query = """
SELECT DISTINCT r.safetyreportid
FROM report r
JOIN drug d ON r.safetyreportid = d.safetyreportid
JOIN drug_activesubstance a ON d.id = a.drug_id
WHERE a.activesubstancename = 'INFLIXIMAB';
"""

cursor.execute(query)
results = cursor.fetchall()
print(f"Found {len(results)} reports with active substance INFLIXIMAB.")


Found 524 reports with active substance INFLIXIMAB.


## Query 6: Count of reactions per report

In [18]:
query = """
SELECT r.safetyreportid, COUNT(re.id) AS reaction_count
FROM report r
JOIN reaction re ON r.safetyreportid = re.safetyreportid
GROUP BY r.safetyreportid
ORDER BY reaction_count DESC;
"""

cursor.execute(query)
results = cursor.fetchall()
print(f"Top report has {results[0][1]} reactions.")
print(f"Found reaction counts for {len(results)} reports.")


Top report has 147 reactions.
Found reaction counts for 36000 reports.
